In [37]:
import numpy as np
import math

# Puma Configuration
Implementation of Inverse and Forward Kinematics

In [38]:
class puma():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.a1=0.25
    self.a2=0.25
    self.a3=0.25

    self.theta1=0
    self.theta2=0
    self.theta3=0

  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()

# Inverse Kinematics
  def invkine(self):
    self.theta1=np.arctan2(self.y,self.x)

    r=math.pow(math.pow(self.x,2)+math.pow(self.y,2),0.5)
    s=self.z-self.a1

    D= (math.pow(r,2)+math.pow(s,2)-math.pow(self.a2,2)-math.pow(self.a3,2)) / (2*self.a2*self.a3)
    if D>1 or D<-1 :
      print("out of the workspace")

    self.theta3 = np.arctan2((-math.pow(1-math.pow(D,2),0.5)),D)
    
    self.theta2 = np.arctan2(s,r)-np.arctan2(self.a3*np.sin(self.theta3),(self.a2+self.a3*np.cos(self.theta3)))

  def dhpara(self):
    mat = []
    mat.append([0,0,self.a1,self.theta1])
    mat.append([self.a2,np.pi/2,0,self.theta2])
    mat.append([self.a3,0,0,self.theta3])

    return mat

  def dh(self,arr):
    a=arr[0]
    alpha=arr[1]
    d=arr[2]
    theta = arr[3]

    mat = []
    b=[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)]
    mat.append(b)

    b=[np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)]
    mat.append(b)

    b=[0, np.sin(alpha), np.cos(alpha), d]
    mat.append(b)

    b=[0,0,0,1]
    mat.append(b)

    return np.matrix(mat)

# Forward Kinematics
  # def forward(self):
  #   matrix=self.dhpara()
  #   T = np.identity(4)
  #   for i in range(len(matrix)):
  #     T=np.matmul(T,self.dh(matrix[i]))
  #   return T[:3,3].T

  def forward(self):
    z1=self.a2*np.sin(self.theta2)+self.a3*np.sin(self.theta2+self.theta3)+self.a1

    r=self.a2*np.cos(self.theta2)+self.a3*np.cos(self.theta2+self.theta3)
    x1=r*np.cos(self.theta1)
    y1=r*np.sin(self.theta1)

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n theta3: {self.theta3} rad \n\n")

In [39]:
puma1 = puma()
cord = [[0.45,0.075,0.1],[0.45,-0.075,0.1],[0.25,-0.075,0.1],[0.25,0.075,0.1]]
lett = ["A","B","C","D"]

for i in range(len(cord)):
  print(lett[i])
  puma1.verify(cord[i])


A
Original Co-ordinates:- [0.45, 0.075, 0.1]

Verified from code :- [0.44999999999999996, 0.07499999999999998, 0.10000000000000003]

Joint parameters: 
 theta1: 0.16514867741462683 rad 
 theta2: -0.03554761311846932 rad 
 theta3: -0.5642311597275824 rad 


B
Original Co-ordinates:- [0.45, -0.075, 0.1]

Verified from code :- [0.44999999999999996, -0.07499999999999998, 0.10000000000000003]

Joint parameters: 
 theta1: -0.16514867741462683 rad 
 theta2: -0.03554761311846932 rad 
 theta3: -0.5642311597275824 rad 


C
Original Co-ordinates:- [0.25, -0.075, 0.1]

Verified from code :- [0.25, -0.07500000000000001, 0.1]

Joint parameters: 
 theta1: -0.2914567944778671 rad 
 theta2: 0.4030874009041333 rad 
 theta3: -1.8493860291865472 rad 


D
Original Co-ordinates:- [0.25, 0.075, 0.1]

Verified from code :- [0.25, 0.07500000000000001, 0.1]

Joint parameters: 
 theta1: 0.2914567944778671 rad 
 theta2: 0.4030874009041333 rad 
 theta3: -1.8493860291865472 rad 




# Stanford Configuration
Implementation of Inverse and Forward Kinematics

In [40]:
class stanford():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.theta1=0
    self.theta2=0
    self.d3=0

    self.a1=0.25
    self.a2=0.25

  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()

# Inverse Kinematics
  def invkine(self):
    if (self.x==0 and self.y==0) :
      print("Singularity !!!!!!!")
      return
    else:
      self.theta1 = np.arctan2(self.y,self.x)

    r= (self.x**2+self.y**2)**(0.5)
    s=self.z-self.a1

    self.theta2 = np.arctan2(s,r)+(np.pi/2)

    self.d3=(r**2+s**2)**(0.5) - self.a2

# Forward Kinematics
  def forward(self):
    x1=(self.a2+self.d3)*np.cos(self.theta1)
    y1=(self.a2+self.d3)*np.sin(self.theta1)
    z1=self.a1+(self.a2+self.d3)*np.sin(self.theta2)

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n d3: {self.d3} m \n\n")


In [41]:
stf=stanford()
cord = [[0.45,0.075,0.1],[0.45,-0.075,0.1],[0.25,-0.075,0.1],[0.25,0.075,0.1]]
lett = ["A","B","C","D"]

for i in range(len(cord)):
  print(lett[i])
  stf.verify(cord[i])


A
Original Co-ordinates:- [0.45, 0.075, 0.1]

Verified from code :- [0.4737002131009568, 0.07895003551682611, 0.7062071897723665]

Joint parameters: 
 theta1: 0.16514867741462683 rad 
 theta2: 1.2531331338126361 rad 
 d3: 0.23023431780746362 m 


B
Original Co-ordinates:- [0.45, -0.075, 0.1]

Verified from code :- [0.4737002131009568, -0.07895003551682611, 0.7062071897723665]

Joint parameters: 
 theta1: -0.16514867741462683 rad 
 theta2: 1.2531331338126361 rad 
 d3: 0.23023431780746362 m 


C
Original Co-ordinates:- [0.25, -0.075, 0.1]

Verified from code :- [0.28834389508859454, -0.08650316852657836, 0.5110076627227638]

Joint parameters: 
 theta1: -0.2914567944778671 rad 
 theta2: 1.0491907131057563 rad 
 d3: 0.051039864469807406 m 


D
Original Co-ordinates:- [0.25, 0.075, 0.1]

Verified from code :- [0.28834389508859454, 0.08650316852657836, 0.5110076627227638]

Joint parameters: 
 theta1: 0.2914567944778671 rad 
 theta2: 1.0491907131057563 rad 
 d3: 0.051039864469807406 m 




# SCARA COnfiguration
Implementation of Inverse and Forward Kinematics

In [42]:
class scara():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.theta1=0
    self.theta2=0
    self.d3=0

    self.a1=0.25
    self.a2=0.25
    self.h=0.25
  
  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()

# Inverse Kinematics
  def invkine(self):
    r= ((self.x**2+self.y**2-self.a1**2-self.a2**2)/(2*self.a1*self.a2))
    self.theta2 = np.arctan2(((1-r**2)**0.5),r)
    self.theta1 = np.arctan2(self.y,self.x)-np.arctan2(self.a2*np.sin(self.theta2),(self.a1+self.a2*np.cos(self.theta2)))

    self.d3=self.h-self.z

# Forward Kinematics
  def forward(self):
    x1=self.a1*np.cos(self.theta1)+self.a2*np.cos(self.theta1+self.theta2)
    y1=self.a1*np.sin(self.theta1)+self.a2*np.sin(self.theta1+self.theta2)
    z1=self.h-self.d3;

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n d3: {self.d3} m \n\n")

In [43]:
scr=scara()
cord = [[0.45,0.075,0.1],[0.45,-0.075,0.1],[0.25,-0.075,0.1],[0.25,0.075,0.1]]
lett = ["A","B","C","D"]

for i in range(len(cord)):
  print(lett[i])
  scr.verify(cord[i])

A
Original Co-ordinates:- [0.45, 0.075, 0.1]

Verified from code :- [0.45, 0.075, 0.1]

Joint parameters: 
 theta1: -0.256502576982114 rad 
 theta2: 0.8433025087934816 rad 
 d3: 0.15 m 


B
Original Co-ordinates:- [0.45, -0.075, 0.1]

Verified from code :- [0.45, -0.075, 0.1]

Joint parameters: 
 theta1: -0.5867999318113676 rad 
 theta2: 0.8433025087934816 rad 
 d3: 0.15 m 


C
Original Co-ordinates:- [0.25, -0.075, 0.1]

Verified from code :- [0.24999999999999994, -0.07500000000000004, 0.1]

Joint parameters: 
 theta1: -1.313041064939743 rad 
 theta2: 2.0431685409237517 rad 
 d3: 0.15 m 


D
Original Co-ordinates:- [0.25, 0.075, 0.1]

Verified from code :- [0.24999999999999994, 0.07500000000000001, 0.1]

Joint parameters: 
 theta1: -0.7301274759840087 rad 
 theta2: 2.0431685409237517 rad 
 d3: 0.15 m 


